In [2]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf
abs_config_dir = os.path.abspath("config/")

with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config = compose(config_name="/oracle/multifidelity.yaml", overrides=[])
    print(OmegaConf.to_yaml(config))
    print(config)


oracle:
  _target_: oracle.oracle.MultiFidelityOracle
  oracles:
  - branin1:
      _target_: gflownet.proxy.box.branin.Branin
      fidelity: 1
      do_domain_map: true
      cost: 100
  - branin01:
      _target_: gflownet.proxy.box.branin.Branin
      fidelity: 0.1
      do_domain_map: true
      cost: 10
  - branin001:
      _target_: gflownet.proxy.box.branin.Branin
      fidelity: 0.01
      do_domain_map: true
      cost: 1

{'oracle': {'_target_': 'oracle.oracle.MultiFidelityOracle', 'oracles': [{'branin1': {'_target_': 'gflownet.proxy.box.branin.Branin', 'fidelity': 1, 'do_domain_map': True, 'cost': 100}}, {'branin01': {'_target_': 'gflownet.proxy.box.branin.Branin', 'fidelity': 0.1, 'do_domain_map': True, 'cost': 10}}, {'branin001': {'_target_': 'gflownet.proxy.box.branin.Branin', 'fidelity': 0.01, 'do_domain_map': True, 'cost': 1}}]}}


In [10]:
# Oracle
from oracle.oracle import HartmannOracle
oracle_1 = HartmannOracle(fidelity=1)
oracle_01 = HartmannOracle(fidelity=0.1)
oracle_001 = HartmannOracle(fidelity=0.01)


# Dataset
from dataset.grid import HartmannDatasetHandler
dataset_handler = HartmannDatasetHandler(
    grid_size=10,
    train_path="./storage/hartmann/data_train.csv",
    train_fraction=1.0,
)

In [6]:
train_x, train_y = dataset_handler.train_data[:]

In [12]:
oracle_1(train_x), oracle_01(train_x), oracle_001(train_x)

(tensor([-0.0976, -0.0510, -0.3727, -0.3996, -0.6481, -0.8401, -0.9133, -1.0907,
         -1.3106, -1.1868, -1.5895, -1.5020, -1.8520, -1.7501, -2.2254, -2.1657,
         -2.5686, -2.4373, -2.8762, -2.7287, -0.1891, -0.1616, -0.0982, -0.0936,
         -0.0423], dtype=torch.float64),
 tensor([-0.0976, -0.0510, -0.3727, -0.3994, -0.6439, -0.8401, -0.9132, -1.0761,
         -1.3058, -1.1764, -1.5736, -1.5019, -1.8279, -1.7501, -2.2152, -2.1653,
         -2.5494, -2.4089, -2.8760, -2.7074, -0.1883, -0.1615, -0.0973, -0.0857,
         -0.0423], dtype=torch.float64),
 tensor([-0.0976, -0.0510, -0.3727, -0.3994, -0.6435, -0.8401, -0.9132, -1.0747,
         -1.3053, -1.1753, -1.5720, -1.5019, -1.8255, -1.7501, -2.2141, -2.1652,
         -2.5475, -2.4060, -2.8760, -2.7053, -0.1883, -0.1615, -0.0972, -0.0849,
         -0.0423], dtype=torch.float64))

In [26]:
import torch
train_x_all = torch.cat([train_x, train_x, train_x])
train_y_all = torch.cat([oracle_1(train_x), oracle_01(train_x), oracle_001(train_x)])
train_y_all = train_y_all.unsqueeze(-1)
train_fid_all = torch.tensor([1]*len(train_x) + [2]*len(train_x) + [3]*len(train_x))
train_fid_all = train_fid_all.unsqueeze(-1)

In [27]:
train_input = torch.cat([train_x_all, train_fid_all], axis=1)
train_input.shape

torch.Size([75, 7])

In [29]:
from botorch.models.gp_regression_fidelity import SingleTaskMultiFidelityGP
import gpytorch
from botorch.models.transforms.outcome import Standardize
from botorch.fit import fit_gpytorch_mll

fid_column = 6
model = SingleTaskMultiFidelityGP(
    train_input,
    train_y_all,
    outcome_transform=Standardize(m=1),
    # fid column
    data_fidelity=fid_column,
)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
mll.to(train_x)
mll = fit_gpytorch_mll(mll)

/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


In [37]:
model.eval()
model.likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    posterior = model.posterior(torch.tensor([[0.0,0.0,0.0,0.0,0.0,0.0,1],[0.0,0.0,0.0,0.0,0.0,0.0,2],[0.0,0.0,0.0,0.0,0.0,0.0,3]]))
    y_mean = posterior.mean
    y_std = posterior.variance.sqrt()

In [38]:
y_mean, y_std

(tensor([[-0.5702],
         [-0.5702],
         [-0.5702]], dtype=torch.float64),
 tensor([[0.8429],
         [0.8429],
         [0.8429]], dtype=torch.float64))

In [39]:
plot_set, _, _ = dataset_handler.get_candidate_set(step=2, as_dataloader=False)


In [51]:
test_fid = torch.tensor([1]*len(plot_set)).unsqueeze(-1)
test_input = torch.cat([plot_set[:], test_fid], axis=-1)

In [52]:
model.eval()
model.likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    posterior = model.posterior(test_input)
    y_mean = posterior.mean
    y_std = posterior.variance.sqrt()

In [53]:
y_mean, y_std

(tensor([[-0.5702],
         [-0.6026],
         [-0.6606],
         ...,
         [-0.7640],
         [-0.5945],
         [-0.5342]], dtype=torch.float64),
 tensor([[0.8429],
         [0.8394],
         [0.8346],
         ...,
         [0.7718],
         [0.8178],
         [0.8377]], dtype=torch.float64))

In [54]:

from botorch.acquisition.max_value_entropy_search import (
    qLowerBoundMaxValueEntropy,
)
acqf = qLowerBoundMaxValueEntropy(
    model,
    candidate_set=test_input,
)
acqf(test_input.unsqueeze(1)).detach()

tensor([0.0011, 0.0010, 0.0007,  ..., 0.0001, 0.0007, 0.0012],
       dtype=torch.float64)